# FinBERT Sentiment Analysis

In [1]:
!git clone https://github.com/AI4Finance-Foundation/FinGPT.git
%cd FinGPT



C:\Users\ARIJIT\Desktop\Final-Year-Project\SentimentAnalysis\FinGPT


Cloning into 'FinGPT'...
C:\Users\ARIJIT\Desktop\Final-Year-Project\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [1]:
%pip install -r requirements.txt
%pip install -q transformers datasets sentencepiece



Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
You should consider upgrading via the 'c:\Users\ARIJIT\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ARIJIT\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
%pip install transformers==4.40.1 peft==0.4.0
%pip install sentencepiece
%pip install accelerate
%pip install torch
%pip install peft
%pip install datasets
%pip install bitsandbytes


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import pipeline

sentiment_model = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone")

c:\Users\ARIJIT\Desktop\Final-Year-Project\.venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from os import cpu_count
print(cpu_count())

12


In [4]:
# Example financial news headline
text = "Apple shares fall after iPhone sales miss expectations"
result = sentiment_model(text)
print(result)


[{'label': 'Negative', 'score': 0.9999992847442627}]


In [5]:
import pandas as pd
import numpy as np

In [6]:
dataframe = pd.read_csv("../Data Collection/data.csv")

dataframe.head()


,Datetime,News,Link
0,2024-01-01,disney marathon how to register and qualify ch...,https://economictimes.indiatimes.com/news/inte...
1,2024-01-01,bharti airtel services to acquire 97 1 stake i...,https://economictimes.indiatimes.com/industry/...
2,2024-01-01,fallout 76 nuke codes launch site locations he...,https://economictimes.indiatimes.com/news/inte...
3,2024-01-01,is retirement on your list in 2024 here are de...,https://economictimes.indiatimes.com/news/inte...
4,2024-01-01,raise a glass to gift city why the recent amen...,https://economictimes.indiatimes.com/opinion/e...


In [7]:
%pip install colorama

Note: you may need to restart the kernel to use updated packages.


In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from colorama import Fore, Style, init
import pandas as pd
import os
import time
import warnings
from transformers import pipeline

init(autoreset=True)

# Global Constants
INPUT_FILE = "../Data Collection/data.csv"
OUTPUT_FILE = "finbert_sentiment_analysis_data.csv"
NUM_PROCESSES = min(4, os.cpu_count() - 1)
MODEL = "yiyanghkust/finbert-tone"

def load_model(model):
  sentiment_model = pipeline("sentiment-analysis", model=model)
  return sentiment_model

sentiment_model = load_model(MODEL)




def find_sentiment(text):
  try:
    text = preprocess(text)
    print(f"{Fore.GREEN} Input:- {text[:20]}{Style.RESET_ALL}")
    result = sentiment_model(text)
    print(f"{Fore.GREEN} Output:- {result}{Style.RESET_ALL}")
    return list((text, result[0]["label"], result[0]["score"]))
  except Exception as e:
    print(f"{Fore.RED} Error: {e}{Style.RESET_ALL}")
    return list((text, np.nan, np.nan))

def process_texts_in_parallel(texts):
    """
        process sentiment from the texts in paprallel
    """
    print(f"{Fore.CYAN}🔹 Using {NUM_PROCESSES} parallel threads...{Style.RESET_ALL}")

    with ThreadPoolExecutor(NUM_PROCESSES) as executor:
        results = list(executor.map(find_sentiment, texts))

    return results


def sentiment_analysis_of_text_data():
    if not os.path.exists(INPUT_FILE):
        print(f"{Fore.RED}Error: '{INPUT_FILE}' not found!{Style.RESET_ALL}")
        return

    df = pd.read_csv(INPUT_FILE)

    if "News" not in df.columns:
        print(f"{Fore.RED}Error: 'News' column missing in CSV!{Style.RESET_ALL}")
        return

    texts = df["News"].dropna().tolist()

    # when the scrapping stops unexpectedly
    already_processed_texts = set()
    if os.path.exists(OUTPUT_FILE):
        already_processed_texts = set(
            pd.read_csv(OUTPUT_FILE)["News"].dropna().tolist()
        )

    # filter the links if they are already processed

    links = [text for text in texts if text not in already_processed_texts]

    # Split into batches to avoid excessive memory usage
    batch_size = 500
    total_batches = (len(texts) // batch_size) + 1

    all_results = []

    for i in range(total_batches):
        batch_texts = texts[i * batch_size: (i + 1) * batch_size]
        print(f"{Fore.YELLOW} Processing batch {i + 1}/{total_batches} ({len(batch_texts)} links){Style.RESET_ALL}")

        results = process_texts_in_parallel(batch_texts)  # Removed `[0]`
        all_results.extend(results)

        # Save intermediate results
        temp_df = pd.DataFrame(all_results, columns=["News", "sentiment_label", "sentiment_score"])
        temp_df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")

    print(f"{Fore.GREEN}✔️ Sentiment Analysis complete! Data saved to '{OUTPUT_FILE}'{Style.RESET_ALL}")

c:\Users\ARIJIT\Desktop\Final-Year-Project\.venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
if __name__ == "__main__":
   sentiment_analysis_of_text_data()

 Processing batch 1/553 (500 links)
🔹 Using 4 parallel threads...
 Input:- disney marathon how 
 Input:- bharti airtel servic Input:- fallout 76 nuke code
 Input:- is retirement on you
 Output:- [{'label': 'Neutral', 'score': 0.9999434947967529}]
 Input:- raise a glass to gif
 Output:- [{'label': 'Neutral', 'score': 0.9980815649032593}] Output:- [{'label': 'Neutral', 'score': 0.9998470544815063}]

 Input:- when psus start to f Output:- [{'label': 'Neutral', 'score': 0.999674916267395}] Input:- india infrantic the 


 Input:- listening to imagine
 Output:- [{'label': 'Neutral', 'score': 0.999075174331665}] Output:- [{'label': 'Positive', 'score': 0.9932195544242859}]

 Input:- demonstrating democr
 Input:- two fresh covid case
 Output:- [{'label': 'Neutral', 'score': 0.999613344669342}]
 Input:- manipur four people 
 Output:- [{'label': 'Neutral', 'score': 0.926517903804779}]
 Input:- isis module case 6 a
 Output:- [{'label': 'Positive', 'score': 1.0}]
 Input:- india inc welcomes 2
 Out

KeyboardInterrupt: 